Key Takewaways

1: Males in their late teens and early twenties make up the vast majority of purchases.  We can continue to lean into this target audience for growth.

2: Repeat purchases are fairly uncommon (i.e., most players have only made a single purchase).  We should be exploring our pricing strategy and the features of the most popular items to explore how to encourage repeat purchase behavior.

3: The most expensive item is $4.99 and least expensive is $1.00.  However, our most popular items are not the least expensive.  We should be exploring a broader portfolio of items at a broader range of price points, creating a more accessible, low price point and a super premium price point.



In [1]:
#import pandas, numpy, matplotlib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
#open and preview gaming data
game_path="Desktop/game.csv"
game_df=pd.read_csv(game_path)
game_df.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [3]:
#count number of unique players using the .nunique() command
print('Total Unique Players: ' + str(game_df['SN'].nunique()))

Total Unique Players: 576


In [4]:
#Create purchasing overview stats dictionary and export to dataframe
purchasing_overview={'Unique Items Purchased' : game_df['Item Name'].nunique(), 
                     'Average Purchase Price': game_df['Price'].mean(),
                     'Number of Purchases': game_df['Purchase ID'].count(),
                     'Total Revenue': game_df['Price'].sum()}
                     
purchasing_overview_df = pd.DataFrame([purchasing_overview], columns=purchasing_overview.keys())

#reformating numbers
purchasing_overview_df["Total Revenue"] = purchasing_overview_df["Total Revenue"].map("${:.2f}".format)
purchasing_overview_df["Average Purchase Price"] = purchasing_overview_df["Average Purchase Price"].map("${:.2f}".format)

purchasing_overview_df



,Unique Items Purchased,Average Purchase Price,Number of Purchases,Total Revenue
0,179,$3.05,780,$2379.77


In [5]:
#Drop duplicates for identifying gender profile of purchases
players_df = game_df.iloc[:,[1,2,3]]
players_df=players_df.drop_duplicates(subset=['SN'],keep='first', inplace=False)
players_df['SN'].count()

576

In [6]:
#Get percentages and counts of gender using value counts, then merge the data frames

gender_proportions=players_df['Gender'].value_counts(normalize=True).reset_index()
gender_proportions=gender_proportions.rename(index=str, columns={"index": "Gender", "Gender": "Percentage"})
gender_counts=players_df['Gender'].value_counts(normalize=False).reset_index()
gender_counts=gender_counts.rename(index=str, columns={"index": "Gender", "Gender": "Count"})

gender_proportions_df = pd.merge(gender_proportions,gender_counts, on='Gender', how='inner')
gender_proportions_df

gender_proportions_df["Percentage"] = gender_proportions_df["Percentage"].map("{:.2f}".format)

#converting gender to the index
gender_proportions_df.set_index('Gender', inplace=True)

gender_proportions_df


,Percentage,Count
Gender,,
Male,0.84,484
Female,0.14,81
Other / Non-Disclosed,0.02,11


In [7]:
#create mean price by gender
g_mean_price_df = game_df.iloc[:,[3,6]]
g_mean_price_df = g_mean_price_df.groupby("Gender").mean()

#create purchase count by gender
g_purchase_count_df = game_df.iloc[:,[3,6]]
g_purchase_count_df = g_purchase_count_df.groupby("Gender").count()

#create purchase sum by gender
g_purchase_sum_df = game_df.iloc[:,[3,6]]
g_purchase_sum_df = g_purchase_sum_df.groupby("Gender").sum()

#create mean purchase by player and by gender
sn_g_purchase_sum_df = game_df.iloc[:,[1,3,6]]
sn_g_purchase_sum_df = sn_g_purchase_sum_df.groupby(["SN","Gender"]).sum()
sn_g_mean_price_df = sn_g_purchase_sum_df.groupby('Gender').mean()

In [8]:
#merge gender data tables, rename headers, format numbers
gender_descriptives_df = pd.merge(g_purchase_count_df,g_mean_price_df, on='Gender', how='inner')
gender_descriptives_df = gender_descriptives_df.rename(index=str, columns={"Price_x": "Purchase Count"
                                                                         , "Price_y": "Average Purchase Price"})
gender_descriptives_df = pd.merge(gender_descriptives_df,sn_g_mean_price_df, on='Gender', how='inner')
gender_descriptives_df = pd.merge(gender_descriptives_df,g_purchase_sum_df, on='Gender', how='inner')
gender_descriptives_df = gender_descriptives_df.rename(index=str, columns={"Price_x": "Average Purchase Price Per Player"
                                                                         , "Price_y": "Total Purchase Volume"})


gender_descriptives_df["Average Purchase Price"] = gender_descriptives_df["Average Purchase Price"].map("${:.2f}".format)
gender_descriptives_df["Total Purchase Volume"] = gender_descriptives_df["Total Purchase Volume"].map("${:.2f}".format)
gender_descriptives_df["Average Purchase Price Per Player"] = gender_descriptives_df["Average Purchase Price Per Player"].map("${:.2f}".format)

gender_descriptives_df


,Purchase Count,Average Purchase Price,Average Purchase Price Per Player,Total Purchase Volume
Gender,,,,
Female,113,$3.20,$4.47,$361.94
Male,652,$3.02,$4.07,$1967.64
Other / Non-Disclosed,15,$3.35,$4.56,$50.19


In [9]:
#Building age bins
bins= [0,9,14,19,24,29,34,39,200]
group_names = ['Younger than 10','10-14','15-19','20-24','25-29','30-34','35-39','40 and older']
players_df['Age Groups'] = pd.cut(players_df['Age'], bins, labels=group_names)
game_df['Age Groups'] = pd.cut(game_df['Age'], bins, labels=group_names)

In [10]:
#Get percentages and counts of age groups using value counts
age_proportions=players_df['Age Groups'].value_counts(normalize=True).reset_index()
age_proportions=age_proportions.rename(index=str, columns={"index": "Age Groups", "Age Groups": "Percentage"})
age_counts=players_df['Age Groups'].value_counts(normalize=False).reset_index()
age_counts=age_counts.rename(index=str, columns={"index": "Age Groups", "Age Groups": "Count"})

#merge data frames
age_proportions_df = pd.merge(age_proportions,age_counts, on='Age Groups', how='inner')

#reformat percentage values
age_proportions_df["Percentage"] = age_proportions_df["Percentage"].map("{:.2f}".format)

#converting gender to the index
age_proportions_df.set_index('Age Groups', inplace=True)

#sorting age groups
age_proportions_df.sort_values('Age Groups')


,Percentage,Count
Age Groups,,
Younger than 10,0.03,17
10-14,0.04,22
15-19,0.19,107
20-24,0.45,258
25-29,0.13,77
30-34,0.09,52
35-39,0.05,31
40 and older,0.02,12


In [11]:
#create mean price by age group
a_mean_price_df = game_df.iloc[:,[6,7]]
a_mean_price_df = a_mean_price_df.groupby("Age Groups").mean()
a_mean_price_df

#create purchase count by age group
a_purchase_count_df = game_df.iloc[:,[6,7]]
a_purchase_count_df = a_purchase_count_df.groupby("Age Groups").count()

#create purchase sum by age group
a_purchase_sum_df = game_df.iloc[:,[6,7]]
a_purchase_sum_df = a_purchase_sum_df.groupby("Age Groups").sum()

#create mean purchase by player and by gender
sn_a_purchase_sum_df = game_df.iloc[:,[1,3,6,7]]
sn_a_purchase_sum_df = sn_a_purchase_sum_df.groupby(["SN","Age Groups"]).sum()
sn_a_mean_price_df = sn_a_purchase_sum_df.groupby("Age Groups").mean()

In [12]:
#merge age data tables, rename headers, format numbers
age_descriptives_df = pd.merge(a_purchase_count_df,a_mean_price_df, on='Age Groups', how='inner')
age_descriptives_df = age_descriptives_df.rename(index=str, columns={"Price_x": "Purchase Count"
                                                                         , "Price_y": "Average Purchase Price"})
age_descriptives_df = pd.merge(age_descriptives_df,sn_a_mean_price_df, on='Age Groups', how='inner')
age_descriptives_df = pd.merge(age_descriptives_df,a_purchase_sum_df, on='Age Groups', how='inner')
age_descriptives_df = age_descriptives_df.rename(index=str, columns={"Price_x": "Average Purchase Price Per Player"
                                                                         , "Price_y": "Total Purchase Volume"})

age_descriptives_df["Average Purchase Price"] = age_descriptives_df["Average Purchase Price"].map("${:.2f}".format)
age_descriptives_df["Average Purchase Price Per Player"] = age_descriptives_df["Average Purchase Price Per Player"].map("${:.2f}".format)
age_descriptives_df["Total Purchase Volume"] = age_descriptives_df["Total Purchase Volume"].map("${:.2f}".format)

age_descriptives_df

,Purchase Count,Average Purchase Price,Average Purchase Price Per Player,Total Purchase Volume
Age Groups,,,,
Younger than 10,23,$3.35,$4.54,$77.13
10-14,28,$2.96,$3.76,$82.78
15-19,136,$3.04,$3.86,$412.89
20-24,365,$3.05,$4.32,$1114.06
25-29,101,$2.90,$3.81,$293.00
30-34,73,$2.93,$4.12,$214.00
35-39,41,$3.60,$4.76,$147.67
40 and older,13,$2.94,$3.19,$38.24


In [13]:
#Creating the individual transactions data frame 
grouped_transactions_df = game_df.groupby('SN').nunique()

#Creating reduced transactions dataset & relabeling
grouped_transactions_df = grouped_transactions_df.iloc[:,[0]]
grouped_transactions_df = grouped_transactions_df.rename(index=str, columns={"Purchase ID": "Number of Transactions"})

In [14]:
#create mean price by age group
p_mean_price_df = game_df.iloc[:,[1,6]]
p_mean_price_df = p_mean_price_df.groupby("SN").mean()

#create purchase count by age group
p_purchase_count_df = game_df.iloc[:,[1,6]]
p_purchase_count_df = p_purchase_count_df.groupby("SN").count()

#create purchase sum by age group
p_purchase_sum_df = game_df.iloc[:,[1,6]]
p_purchase_sum_df = p_purchase_sum_df.groupby("SN").sum()

In [15]:
#merge player data tables, rename headers, format numbers
player_descriptives_df = pd.merge(p_purchase_count_df,p_mean_price_df, on='SN', how='inner')
player_descriptives_df = pd.merge(player_descriptives_df,p_purchase_sum_df, on='SN', how='inner')
player_descriptives_df = player_descriptives_df.rename(index=str, columns={"Price_x": "Purchase Count"
                                                                         , "Price_y": "Average Purchase Price"
                                                                        , "Price":"Total Purchase Amount"})

#sorting age groups
player_descriptives_df=player_descriptives_df.sort_values(by='Total Purchase Amount', ascending=False)

#Reformatting
player_descriptives_df["Average Purchase Price"] = player_descriptives_df["Average Purchase Price"].map("${:.2f}".format)
player_descriptives_df["Total Purchase Amount"] = player_descriptives_df["Total Purchase Amount"].map("${:.2f}".format)

player_descriptives_df.head(10)


,Purchase Count,Average Purchase Price,Total Purchase Amount
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10
Ilarin91,3,$4.23,$12.70
Ialallo29,3,$3.95,$11.84
Tyidaim51,3,$3.94,$11.83
Lassilsala30,3,$3.84,$11.51


In [16]:
#create mean price by age group
gm_mean_price_df = game_df.iloc[:,[4,6]]
gm_mean_price_df = gm_mean_price_df.groupby("Item ID").mean()

#create purchase count by age group
gm_purchase_count_df = game_df.iloc[:,[4,6]]
gm_purchase_count_df = gm_purchase_count_df.groupby("Item ID").count()

#create purchase sum by age group
gm_purchase_sum_df = game_df.iloc[:,[4,6]]
gm_purchase_sum_df = gm_purchase_sum_df.groupby("Item ID").sum()

In [17]:

items_table = game_df.drop_duplicates(['Item ID'])
items_table = items_table.iloc[:,[4,5]]
items_table.reset_index()

#merge player data tables, rename headers, format numbers
game_descriptives_df = pd.merge(gm_purchase_count_df,gm_mean_price_df, on='Item ID', how='inner')
game_descriptives_df = pd.merge(game_descriptives_df,gm_purchase_sum_df, on='Item ID', how='inner')
game_descriptives_df = pd.merge(game_descriptives_df,items_table,on='Item ID', how='inner')
game_descriptives_df = game_descriptives_df.rename(index=str, columns={"Price_x": "Purchase Count"
                                                                         , "Price_y": "Purchase Price"
                                                                        , "Price":"Total Revenue Amount"})

#sorting
game_descriptives_df=game_descriptives_df.sort_values(by='Purchase Count', ascending=False)
game_descriptives_df = game_descriptives_df[['Item ID', 'Item Name', 'Purchase Count', 'Purchase Price', 'Total Revenue Amount']]




game_descriptives_df.head(10)

,Item ID,Item Name,Purchase Count,Purchase Price,Total Revenue Amount
177,178,"Oathbreaker, Last Hope of the Breaking Storm",12,4.23,50.76
144,145,Fiery Glass Crusader,9,4.58,41.22
107,108,"Extraction, Quickblade Of Trembling Hands",9,3.53,31.77
81,82,Nirvana,9,4.90,44.10
19,19,"Pursuit, Cudgel of Necromancy",8,1.02,8.16
102,103,Singed Scalpel,8,4.35,34.80
74,75,Brutality Ivory Warmace,8,2.42,19.36
71,72,Winter's Bite,8,3.77,30.16
59,60,Wolf,8,3.54,28.32
58,59,"Lightning, Etcher of the King",8,4.23,33.84


In [18]:
game_descriptives_profits=game_descriptives_df.sort_values(by='Total Revenue Amount', ascending=False)
game_descriptives_profits.head(10)

#Reformatting
#game_descriptives_profits["Total Revenue Amount"] = game_descriptives_profits["Total Revenue Amount"].map("${:.2f}".format)
#game_descriptives_profits["Purchase Price"] = game_descriptives_profits["Purchase Price"].map("${:.2f}".format)

,Item ID,Item Name,Purchase Count,Purchase Price,Total Revenue Amount
177,178,"Oathbreaker, Last Hope of the Breaking Storm",12,4.23,50.76
81,82,Nirvana,9,4.90,44.10
144,145,Fiery Glass Crusader,9,4.58,41.22
91,92,Final Critic,8,4.88,39.04
102,103,Singed Scalpel,8,4.35,34.80
58,59,"Lightning, Etcher of the King",8,4.23,33.84
107,108,"Extraction, Quickblade Of Trembling Hands",9,3.53,31.77
77,78,"Glimmer, Ender of the Moon",7,4.40,30.80
71,72,Winter's Bite,8,3.77,30.16
59,60,Wolf,8,3.54,28.32


In [20]:
game_descriptives_df['Purchase Price'].max()

4.99

In [21]:
game_descriptives_df['Purchase Price'].min()

1.0